## Bug report classification

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from time import time
import matplotlib.pyplot as plt
import sklearn.preprocessing
import os,re

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ping-dev04/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Build data

In [3]:
data = pd.read_csv('idata.csv')

In [4]:
data.head()

Bug ID              Product        Component  \
0  1436291.0  Firefox for Android  Activity Stream   
1  1436379.0  Firefox for Android  Activity Stream   
2  1254660.0  Firefox for Android  Activity Stream   
3  1273667.0  Firefox for Android  Activity Stream   
4  1299685.0  Firefox for Android  Activity Stream   

                                             Summary  
0  Context menu animation broken when "Delete fro...  
1  Links do not work when Pocket stories are upda...  
2  Don't list both http and https versions of a U...  
3  Simplify long-press context menu for top sites...  
4  Consider replacing ActivityStream topsites' Vi...

In [5]:
train = pd.DataFrame()
train['bugID'] = data['Bug ID']
train['text'] = data['Summary']

In [6]:
train.head()

bugID                                               text
0  1436291.0  Context menu animation broken when "Delete fro...
1  1436379.0  Links do not work when Pocket stories are upda...
2  1254660.0  Don't list both http and https versions of a U...
3  1273667.0  Simplify long-press context menu for top sites...
4  1299685.0  Consider replacing ActivityStream topsites' Vi...

--------------

### TFIDF Vectorization

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf_vectorizer = TfidfVectorizer(analyzer="word",
                                   tokenizer=nltk.word_tokenize,
                                   preprocessor=None,
                                   stop_words='english',
                                   max_features=None)
sparce_tfidf = tfidf_vectorizer.fit_transform(train['text'])

In [9]:
from sklearn.decomposition import TruncatedSVD

In [10]:
svd = TruncatedSVD(n_components=100, n_iter=25, random_state=12)
t_tfidf = svd.fit_transform(sparce_tfidf)

-----------------

### GMM

In [11]:
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import silhouette_score

In [12]:
from IPython.display import display

In [13]:
# method to find the best n_components
def score_cluster(data, num_components):
    clusterer = GMM(n_components=num_components)
    clusterer.fit(data)
    preds = clusterer.predict(data)
    score = silhouette_score(data, preds)
    return score

In [14]:
print ("Silhouette Score for different sizes")
silhouette_scores_matrix = pd.DataFrame(index=['Score'])

Silhouette Score for different sizes


In [15]:
for size in range(2,11):
    silhouette_scores_matrix[size] = \
    pd.Series(score_cluster(t_tfidf, size),\
    index = silhouette_scores_matrix.index)

In [16]:
display(silhouette_scores_matrix)

best_n_components = 3

# Apply the selected clustering algorithm to the reduced data 
clusterer = GMM(n_components=best_n_components)
clusterer.fit(t_tfidf)

# Predict the cluster for each data point
preds = clusterer.predict(t_tfidf)

# Find the cluster centers
centers = clusterer.means_

# Predict the cluster for each transformed sample data point
sample_preds = clusterer.predict(t_tfidf[0:10])

# Calculate the mean silhouette coefficient for the number of clusters chosen
score = silhouette_score(t_tfidf, preds)

print("Best Score: {}, n_components={}".format(score, best_n_components))

2         3         4         5         6        7         8   \
Score -0.049044 -0.030824 -0.013987 -0.047036 -0.074541 -0.05676 -0.051063   

             9         10  
Score -0.038443 -0.084138

Best Score: -0.0855640793649877, n_components=3


In [17]:
silhouette_scores_matrix.plot(kind = "barh",figsize=(16,4))

### Result

In [24]:
sample_preds

array([0, 1, 1, 0, 1, 1, 2, 0, 0, 1])

In [19]:
clrs = []
for item in preds:
    if item ==0:
        clrs.append('red')
    elif item ==1:
        clrs.append('blue')
    elif item ==2:
        clrs.append('green')
        

### Severity sclae is : 0,1,2

In [20]:
resultdf = train.copy(deep =True)

In [21]:
resultdf['label'] = preds

In [22]:
resultdf.head(100)

bugID                                               text  label
0   1436291.0  Context menu animation broken when "Delete fro...      0
1   1436379.0  Links do not work when Pocket stories are upda...      1
2   1254660.0  Don't list both http and https versions of a U...      1
3   1273667.0  Simplify long-press context menu for top sites...      0
4   1299685.0  Consider replacing ActivityStream topsites' Vi...      1
5   1315197.0   Activity Stream topsites: fix default announc...      1
6   1319485.0  Add domains table, with domain_id FK in histor...      2
7   1320803.0  Ability to store/retrieve custom name for a To...      0
8   1320806.0  Allow removing Top Sites items without removin...      0
9   1322501.0  Asynchronously obtain, send, and store image d...      1
10  1322564.0            Consider loading icons from the network      1
11  1324452.0  Context menu is misplaced and wrongly rendered...      0
12  1333545.0  Box/Border around items in NavigationView on A...      0
13  1335817.0   Highlights algorithm: Add domain visit frequency      2
14  1335819.0          Highlights algorithm: Use main image size      0
15  1335824.0  Highlights algorithm: Consider lowering score ...      0
16  1338561.0  An incomplete preview context menu is displaye...      0
17  1340541.0  Add toast when user chooses to Copy Address fr...      0
18  1350886.0      Remove MOZ_ANDROID_ACTIVITY_STREAM build flag      1
19  1352342.0             Screen is hard to use on large tablets      1
20  1377292.0                                Unify context menus      0
21  1386725.0  Remove old top sites code once the new Activit...      1
22  1388187.0                Highlights query is still very slow      1
23  1388392.0                Top Sites: Show next page partially      1
24  1388415.0   Investigate how big highlight preview images are      1
25  1388558.0  Default top-sites background is the same as th...      1
26  1389200.0  Add "Save to Pocket" to context menu options f...      0
27  1389259.0  Use screenshots for pages in Top Sites for low...      1
28  1390257.0  Default "letter" favicons font size seems too ...      1
29  1390970.0   Add Pocket entry point if Pocket isn't installed      0
..        ...                                                ...    ...
70  1038960.0  Unable to pin top site: foreign key constraint...      0
71  1314902.0                     Remove duplicated vector icons      1
72  1401519.0   Top Sites area is still scrollable after reac...      0
73  1406046.0  Intermittent testActivityStreamPocketReferrer ...      1
74  1332621.0  Duplicate top sites due to multiple redirects ...      2
75  1400824.0                     Can't reorder the pinned sites      0
76  1390368.0  Intermittent testActivityStreamContextMenu | T...      1
77  1308582.0  Disabling installed Add-ons does not work prop...      1
78  1233548.0  Firefox for Android shows dialogs with "The ad...      0
79   841793.0  We should display add-ons that don't work on p...      1
80   887201.0  No obvious way to update extensions in Firefox...      1
81   895032.0  Do not initialize addon-manager related compon...      1
82   946617.0  Adding an add-on from our addons site doesn't ...      1
83  1010752.0  Add-ons Manager becomes horizontally scrollabl...      0
84  1049864.0  Investigate why the test for bug 687194 fails ...      1
85  1053402.0         UX work - Surface addons in browser chrome      1
86  1063245.0  Unable to copy or paste in add-on settings wit...      1
87  1108763.0  Lightweight themes are disabled in add-on mana...      1
88  1173896.0      No UI to explain disabled blocklisted add-ons      1
89  1194125.0  Show warning in add-on manager list for add-on...      1
90  1196203.0  Re-opening an add-on page from the history mig...      2
91  1197535.0  Installing the same extension multiple times w...      0
92  1202379.0  Firefox for Android should be clearer when fai...      0
93  1225969.0    Add a way to install addons fro

In [23]:
resultdf.groupby(['label']).count()

bugID  text
label             
0       1580  1644
1       3332  3427
2       1035  1083

----------